In [74]:
import numpy as np
from collections import Counter

def data_loader():
    # load data
    with open('train','r') as file:
        train_data = file.read().split('\n')[:-1]
    with open('test','r') as file:
        test_data = file.read().split('\n')[:-1]
    return train_data, test_data

def parser(datum):
    # extract labels and words
    email_addr, label, words = datum.split(' ',2)
    words = words.split()
    # transform words into dictionary
    word_dict = dict(zip([words[i] for i in range(0, len(words), 2)], [int(words[i+1]) for i in range(0, len(words), 2)]))
    # transform label into 0, 1
    if label == 'ham':
        label = 0
    elif label == 'spam':
        label = 1
    else: 
        raise ValueError
    return label, word_dict

def data_preprocessing(train_data, test_data):
    y_train = np.zeros(len(train_data))
    y_test = np.zeros(len(test_data))
    x_train = []
    x_test = []
    for i, datum in enumerate(train_data):
        label, word_dict = parser(datum)
        y_train[i] = label
        x_train.append(word_dict)
    for i, datum in enumerate(test_data):
        label, word_dict = parser(datum)
        y_test[i] = label
        x_test.append(word_dict)
    return x_train, y_train, x_test, y_test

def compute_empirical_pmf_y(y_train):
    # compte distribution P(y=1), P(y=0)
    # TODO
    return np.sum(y_train == 1) / len(y_train),  np.sum(y_train == 0) / len(y_train)

def m_estimation_conditional_probability(x_train_frt, y_train, num_vocab, m):
    # compute P(x_j|y=1) and P(x_j|y=0) for j = 1, ..., d
    # TODO
    p_on_spam = np.zeros((2, x_train_frt.shape[1]))
    p_on_ham = np.zeros((2, x_train_frt.shape[1]))
    
    temp_df = pd.DataFrame(np.hstack([x_train_frt,y_train.reshape(-1,1)]))
    temp_df.columns = temp_df.columns = [*list(temp_df.columns)[:-1],"label"]
    
    for j in range(num_vocab):
        column_data_0 = (temp_df[temp_df["label"]==0])[j]
        column_data_1 = (temp_df[temp_df["label"]==1])[j]
        
        # P(x_j = 1|y=0) by definition
        p_on_ham[1, j] = (sum(column_data_0 > 0) + m) / (len(column_data_0) + m * num_vocab)
        # P(x_j = 0|y=0) by the fact that conditional probability is a valid pmf
        p_on_ham[0, j] = 1 - p_on_ham[1, j]
                                                          
        

        # P(x_j = 1|y=1)
        p_on_spam[1, j] = (sum(column_data_1 > 0) + m) / (len(column_data_1) + m * num_vocab)
        # P(x_j = 0|y=1)
        p_on_spam[0, j] = 1 - p_on_spam[1, j]
                                                          
    return p_on_spam, p_on_ham
                                                          

def log_estimated_probability(p_spam, p_ham, p_on_spam_m, p_on_ham_m, x_frts):
    # compute log(P(y=1, x_1, x_2,..., x_n)) and log(P(y=0, x_1, x_2,..., x_n))
    # hint: log(P(y, x_1, x_2,..., x_n)) = log(P(y=1)) + log(P(x_1 | y)) + ... + log(P(x_d | y))
    # TODO
    
    log_p_spam, log_p_ham = np.zeros((len(x_frts), )), np.zeros((len(x_frts), ))
    
    for i in range(len(x_frts)):
        log_p_spam[i] = np.sum([np.log(p_spam)]+[np.log(p_on_spam_m[1 if x_frts[i, j] > 0 else 0 ,j]) for j in range(x_frts.shape[1])])
        log_p_ham[i] = np.sum([np.log(p_ham)]+[np.log(p_on_ham_m[1 if x_frts[i, j] > 0 else 0 ,j]) for j in range(x_frts.shape[1])])
    
    return log_p_spam,log_p_ham

def accuarcy(y_true, y_pred):
    # calculate accuracy
    # TODO 
    
    return np.sum(y_true == y_pred) / len(y_true)

## Load and preprocess data

In [75]:
from sklearn.feature_extraction import DictVectorizer
# load data
train_data, test_data = data_loader()

# extract labels to 0,1 and features to dicticnary
x_train, y_train, x_test, y_test = data_preprocessing(train_data, test_data)

# transform word dicts to feature vectors
vectorizer = DictVectorizer(sparse=False)
x_train_frt = vectorizer.fit_transform(x_train)
x_test_frt = vectorizer.transform(x_test)
print(x_train_frt.shape, x_test_frt.shape)

(9000, 1000) (1000, 1000)


### Grid search for the best $m$. 

Which $m$ result in the highest accuracy in the test set?

In [76]:
def pipeline(x_train_frt, y_train, x_test_frt, y_test, m):
    p_spam, p_ham = compute_empirical_pmf_y(y_train)
    p_on_spam_m, p_on_ham_m = m_estimation_conditional_probability(x_train_frt, y_train, x_train_frt.shape[1], m)
    log_p_spam, log_p_ham = log_estimated_probability(p_spam, p_ham, p_on_spam_m, p_on_ham_m, x_test_frt)
    test_pred = (log_p_spam > log_p_ham)
    print(str(m) + ":" + str(accuarcy(y_test, test_pred)))

m_grid = [0, 0.01, 0.1, 1, 10, 100, 1000]
for m in m_grid:
    pipeline(x_train_frt, y_train, x_test_frt, y_test, m)

C:\Users\alexm\AppData\Local\Temp\ipykernel_112388\999649774.py:83: RuntimeWarning: divide by zero encountered in log
  log_p_spam[i] = np.sum([np.log(p_spam)]+[np.log(p_on_spam_m[1 if x_frts[i, j] > 0 else 0 ,j]) for j in range(x_frts.shape[1])])


0:0.932
0.01:0.927
0.1:0.924
1:0.922
10:0.905
100:0.88
1000:0.845
